In [ ]:
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
import SimpleITK as sitk
import six
import sys, os

## Feature extractor

In [ ]:
class Feature_Extraction(object):
    def __init__(self, dataDir):
        imageName, maskName = getTestCase('brain1', dataDir)
        self.image = sitk.ReadImage(imageName)
        self.mask = sitk.ReadImage(maskName)    
    
    def Feature_Calculation(self, FeatureObject, FeatureDict):
        FeatureObject.enableAllFeatures() # On the feature class level, all features are disabled by default.
        FeatureObject.calculateFeatures()        
        for (key,val) in six.iteritems(FeatureObject.featureValues):
            FeatureDict[key] = val            
        return FeatureDict
    

## Texture features

In [ ]:
class Text_features(Feature_Extraction, object):
    def Get_Features(self):
        TextFeatures = {}
        FirstOrderFeatures = firstorder.RadiomicsFirstOrder(self.image,self.mask)
        TextFeatures = self.Feature_Calculation(FirstOrderFeatures, TextFeatures)
        del FirstOrderFeatures
        
        ShapeFeatures = shape.RadiomicsShape(self.image, self.mask)
        TextFeatures = self.Feature_Calculation(ShapeFeatures, TextFeatures)
        del ShapeFeatures
        
        GLCM_Features = glcm.RadiomicsGLCM(self.image,self.mask)
        TextFeatures = self.Feature_Calculation(GLCM_Features, TextFeatures)
        del GLCM_Features
        
        GLSZM_Features = glszm.RadiomicsGLSZM(self.image, self.mask)
        TextFeatures = self.Feature_Calculation(GLSZM_Features, TextFeatures)
        del GLSZM_Features
        
        GLRLM_Features = glrlm.RadiomicsGLRLM(self.image, self.mask)
        TextFeatures = self.Feature_Calculation(GLRLM_Features, TextFeatures)
        del GLRLM_Features
        
        return TextFeatures
       

In [ ]:
FeatureList = []
dataDir = '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat138_0001/VSD.Brain_3more.XX.O.OT.42309/VSD.Brain_3more.XX.O.OT.42309.mha'
feature = Text_features(dataDir)
TextFeatures = feature.Get_Features()
FeatureList.append(TextFeatures)
del feature, TextFeatures
dataDir2 = '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat153_0109/VSD.Brain_3more.XX.O.OT.42325/VSD.Brain_3more.XX.O.OT.42325.mha'
feature = Text_features(dataDir2)
TextFeatures = feature.Get_Features()
FeatureList.append(TextFeatures)
del feature, TextFeatures

DF = pd.DataFrame.from_dict(FeatureList)
print(DF)

## Wavelet features

In [ ]:
import pywt
class Wavelet_features(Feature_Extraction, object):
    def Coiflet_wavelet(self):
        self.wavelet = pywt.Wavelet('coif1')
        print(self.wavelet)


## Logging

In [ ]:
import radiomics
import logging
log_file = '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat138_0001/log_file.txt'
handler = logging.FileHandler(filename=log_file, mode='w')  # overwrites log_files from previous runs. Change mode to 'a' to append.
formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # format string for log messages
handler.setFormatter(formatter)
radiomics.logger.addHandler(handler)

# Control the amount of logging stored by setting the level of the logger. N.B. if the level is higher than the
# Verbositiy level, the logger level will also determine the amount of information printed to the output
radiomics.logger.setLevel(logging.DEBUG)

## Clinical Data

In [ ]:
import pandas as pd
DataDir = "/home/seonwhee/Deep_Learning/Datasets/TCGA-GBM"
PatDF = pd.DataFrame.from_csv(path="%s/nationwidechildrens.org_clinical_patient_gbm.txt"%(DataDir), sep="\t", index_col=1, header=[0])
DrugDF = pd.DataFrame.from_csv(path="%s/nationwidechildrens.org_clinical_drug_gbm.txt"%(DataDir), sep="\t")

## Summarize the output features

In [74]:
import pandas as pd
from glob import glob
tsvfiles = glob("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/*/*more*/Table.tsv")
FeatureList = []

for tsv in tsvfiles:
    colname = (tsv.split("/")[-2])
    DF = pd.DataFrame.from_csv(tsv, sep='\t')
    DF.columns = ['Image type', 'Feature Class', 'Feature Name', colname]
    
    DF = DF.reset_index()
    DF = DF.drop('Label', axis=1)
   
    FeatureList.append(DF)

TotalDF = pd.concat(FeatureList, axis=1)
TotalDF = TotalDF.T.drop_duplicates().T
TotalDF.to_csv("/home/seonwhee/Deep_Learning/MRImage_Pipeline/Variable_Selection/RadiomicFeatures.csv", sep="\t", encoding='utf-8')


/home/seonwhee/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
